[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CLDiego/SPE_GeoHackathon_2025/blob/dev/S1_M2_ChatAgent.ipynb)

***
- <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/write.svg" width="20"/> Follow along by running each cell in order
- <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> Make sure to run the environment setup cells first
- <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/reminder.svg" width="20"/> Wait for each installation to complete before proceeding
- <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/list.svg" width="20" /> Don't worry if installations take a while - this is normal!

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Environment setup
!pip -q install langchain langchain-core langchain-community langchain-huggingface torch gradio
!pip -q install bitsandbytes==0.46.0 transformers==4.48.3 

In [ ]:
# Hugging Face API token
# Retrieving the token is required to get access to HF hub
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

# Session 01 // Module 02: First Chat Agent with LangChain

In this module, we'll build our first conversational AI agent using modern LangChain. We'll create a geoscience-focused chatbot that can answer questions about geology, geophysics, and petroleum engineering concepts.

## Learning Objectives
- Understand modern LangChain fundamentals (LCEL, ChatModels, Messages)
- Build a simple Q&A chat agent with Hugging Face models
- Add conversational memory to maintain context
- Create an interactive Gradio interface
- Apply the agent to geoscience conversations

## 1. Modern LangChain Basics

**LangChain** has evolved significantly. Modern LangChain uses:
- **LCEL (LangChain Expression Language)**: Declarative way to compose chains
- **ChatModels**: Specialized for conversational AI
- **Messages**: Structured conversation format
- **Runnables**: Standardized interface for all components
- **Memory**: More flexible conversation state management

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch

### 1.1 Setting up the Modern Language Model

Let's use a modern approach with ChatModels:

In [ ]:
# Use a more conversational model
model_name = "meta-llama/Llama-3.1-8B-Instruct"
model_name = "microsoft/Phi-3-mini-4k-instruct"

# Create HuggingFace pipeline
# Steps:
# 1. Load tokenizer
# 2. Create quantization config
# 3. Create prompt model
tokenizer = AutoTokenizer.from_pretrained(model_name)

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quant_config)

# Set pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Create text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150,
    temperature=0.2,
    do_sample=True, # Sampling enables more diverse outputs
    pad_token_id=tokenizer.eos_token_id,
    return_full_text=False # The generated text will not include the prompt
)

# Create LangChain LLM
llm = HuggingFacePipeline(pipeline=pipe)

# Wrap with ChatHuggingFace for modern interface
chat_model = ChatHuggingFace(llm=llm)

print(f"Model loaded: {model_name}")
print(f"Model parameters: {model.num_parameters():,}")

### 1.2 Creating Modern Prompt Templates

Modern LangChain uses ChatPromptTemplate with structured messages:

In [ ]:
# Create a system prompt for geoscience expertise
# The system prompt sets the behavior and personality of the assistant
system_prompt = """
You are Dr. GeoBot, an expert geophysicist and petroleum engineer with 20 years of experience.
You specialize in seismic interpretation, reservoir characterization, and hydrocarbon exploration.

Guidelines:
- Provide accurate, helpful answers about geoscience topics
- Keep responses concise but informative (2-3 sentences)
- Use technical terms but explain them when needed
- Focus on practical applications and formulas
- If unsure, acknowledge limitations
"""

# Create chat prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])

# Test the template
test_question = "What is porosity?"
formatted_prompt = prompt_template.format_messages(question=test_question)
print("Formatted prompt:")
for message in formatted_prompt:
    print(f"{message.type}: {message.content}")

### 1.3 Creating Modern Chains with LCEL

Modern LangChain uses LCEL (LangChain Expression Language) for composing chains:

In [ ]:
# Create a simple chain using LCEL
simple_chain = prompt_template | chat_model | StrOutputParser()

# Test the chain
print("=== Testing Simple Chain ===")
response = simple_chain.invoke({"question": "What is the difference between porosity and permeability?"})
print(f"Response: {response}")

1.3.1 Low-level API

In [ ]:
from transformers import TextStreamer

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

full_prompt = ""
for msg in formatted_prompt:
    if msg.type == "system":
        full_prompt += f"[SYSTEM]\n{msg.content}\n"
    elif msg.type == "human":
        full_prompt += f"[USER]\n{msg.content}\n"

inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

# Stream tokens as they are generated
model.generate(**inputs, streamer=streamer, max_new_tokens=200)

In [ ]:
# Test with multiple geoscience questions
test_questions = [
    "What is seismic resolution?",
    "How do P-waves differ from S-waves?",
    "What factors affect hydrocarbon migration?"
]

print("=== Testing Multiple Questions ===")
for i, question in enumerate(test_questions, 1):
    print(f"\n{i}. Question: {question}")
    response = simple_chain.invoke({"question": question})
    print(f"   Answer: {response}")
    print("-" * 80)

## 2. Adding Modern Conversational Memory

Modern LangChain uses RunnableWithMessageHistory for conversation management:

In [ ]:
# Create conversational prompt template with history
conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

# Create the conversational chain
conversational_chain = conversational_prompt | chat_model | StrOutputParser()

# Store for conversation histories
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create conversational chain with memory
conversational_with_memory = RunnableWithMessageHistory(
    conversational_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)

print("Conversational chain with memory created!")

In [ ]:
# Test conversational memory
print("=== Testing Conversational Memory ===")

session_config = {"configurable": {"session_id": "test_session"}}

# First question
response1 = conversational_with_memory.invoke(
    {"question": "What is seismic inversion?"},
    config=session_config
)
print(f"Q1: What is seismic inversion?")
print(f"A1: {response1}")
print()

# Follow-up question that refers to previous context
response2 = conversational_with_memory.invoke(
    {"question": "What are the main types of this technique?"},
    config=session_config
)
print(f"Q2: What are the main types of this technique?")
print(f"A2: {response2}")
print()

# Another follow-up
response3 = conversational_with_memory.invoke(
    {"question": "Which type is most commonly used in the industry?"},
    config=session_config
)
print(f"Q3: Which type is most commonly used in the industry?")
print(f"A3: {response3}")
print()

# Check memory content
print("=== Current Memory ===")
history = get_session_history("test_session")
for message in history.messages:
    print(f"{message.type}: {message.content[:100]}...")

## 3. Building a Modern Chat Agent Class

Let's create a robust chat agent using modern LangChain patterns:

In [ ]:
from typing import Dict, Any
import uuid

class ModernGeoscienceChatAgent:
    def __init__(self, chat_model):
        self.chat_model = chat_model
        self.store = {}
        
        # Enhanced system prompt
        self.system_prompt = """
You are Dr. GeoBot, a friendly and knowledgeable geoscience expert specializing in:
- Geophysics and seismic interpretation
- Petroleum geology and reservoir engineering  
- Well logging and formation evaluation
- Hydrocarbon exploration and production
- Geomechanics and drilling engineering

Guidelines:
- Provide accurate, helpful answers about geoscience topics
- Use technical terms but explain them when needed
- Be conversational and engaging
- Keep responses focused and informative
- If unsure, acknowledge limitations honestly
- Reference previous conversation when relevant
"""
        
        # Create prompt template
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{question}")
        ])
        
        # Create chain
        self.chain = self.prompt | self.chat_model | StrOutputParser()
        
        # Create conversational chain with memory
        self.conversational_chain = RunnableWithMessageHistory(
            self.chain,
            self.get_session_history,
            input_messages_key="question",
            history_messages_key="history",
        )
    
    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        return self.store[session_id]
    
    def chat(self, question: str, session_id: str = "default") -> str:
        """Process a question and return a response"""
        try:
            config = {"configurable": {"session_id": session_id}}
            response = self.conversational_chain.invoke(
                {"question": question},
                config=config
            )
            return response.strip()
        except Exception as e:
            return f"I apologize, but I encountered an error: {str(e)}"
    
    def clear_memory(self, session_id: str = "default"):
        """Clear conversation history for a session"""
        if session_id in self.store:
            self.store[session_id].clear()
    
    def get_history(self, session_id: str = "default") -> list:
        """Get conversation history for a session"""
        if session_id in self.store:
            return self.store[session_id].messages
        return []
    
    def create_new_session(self) -> str:
        """Create a new conversation session"""
        return str(uuid.uuid4())

# Create the modern chat agent
chat_agent = ModernGeoscienceChatAgent(chat_model)
print("Modern GeoscienceChatAgent created successfully!")

In [ ]:
# Test the modern chat agent
print("=== Testing Modern GeoscienceChatAgent ===")

# Test conversation
questions = [
    "Hello! Can you explain what you specialize in?",
    "What is the difference between conventional and unconventional reservoirs?",
    "How do geophysicists use seismic data to find oil?",
    "What role does well logging play in this process?"
]

session_id = chat_agent.create_new_session()
print(f"Created session: {session_id[:8]}...\n")

for i, question in enumerate(questions, 1):
    print(f"{i}. Human: {question}")
    response = chat_agent.chat(question, session_id)
    print(f"   Dr. GeoBot: {response}")
    print("-" * 100)

## 4. Creating a Modern Gradio Interface

Let's create an improved web interface:

In [ ]:
import gradio as gr
from typing import List, Tuple

# Create a new chat agent for the interface
gradio_agent = ModernGeoscienceChatAgent(chat_model)

# Global session management
current_session = gradio_agent.create_new_session()

def respond(message: str, history: List[Tuple[str, str]]) -> Tuple[str, List[Tuple[str, str]]]:
    """
    Process user message and return bot response
    """
    global current_session
    
    if not message.strip():
        return "", history
    
    # Get response from agent
    bot_response = gradio_agent.chat(message, current_session)
    
    # Add to chat history
    history.append((message, bot_response))
    
    return "", history

def clear_conversation() -> List[Tuple[str, str]]:
    """
    Clear conversation history and start new session
    """
    global current_session
    gradio_agent.clear_memory(current_session)
    current_session = gradio_agent.create_new_session()
    return []

def load_example(example: str) -> str:
    """
    Load example question into the textbox
    """
    return example

# Create modern Gradio interface
with gr.Blocks(
    title="Dr. GeoBot - Advanced Geoscience Chat Assistant",
    theme=gr.themes.Soft()
) as demo:
    
    gr.Markdown("""
    # 🌍 Dr. GeoBot - Your Advanced Geoscience Expert
    
    I'm an AI geoscience expert powered by modern LangChain. Ask me about:
    
    | **Geophysics** | **Petroleum Engineering** | **Well Logging** |
    |---|---|---|
    | Seismic interpretation | Reservoir characterization | Formation evaluation |
    | Gravity & magnetics | Hydrocarbon systems | Petrophysics |
    | Electromagnetics | Production optimization | Log analysis |
    
    💡 *I remember our conversation, so feel free to ask follow-up questions!*
    """)
    
    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                value=[],
                height=500,
                show_label=False,
                bubble_full_width=False
            )
            
            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Ask me about geoscience topics...",
                    show_label=False,
                    scale=4,
                    container=False
                )
                send_btn = gr.Button("Send 📤", scale=1, variant="primary")
            
            with gr.Row():
                clear_btn = gr.Button("🗑️ Clear Chat", variant="secondary")
                
        with gr.Column(scale=1):
            gr.Markdown("### 💡 Example Questions")
            
            example_questions = [
                "What is seismic inversion?",
                "Explain porosity vs permeability",
                "How do P-waves and S-waves differ?",
                "What is reservoir characterization?",
                "How does well logging work?",
                "What are the challenges in unconventional reservoirs?"
            ]
            
            for question in example_questions:
                example_btn = gr.Button(
                    question,
                    variant="secondary",
                    size="sm"
                )
                example_btn.click(
                    load_example,
                    inputs=[gr.State(question)],
                    outputs=msg
                )
    
    # Event handlers
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    send_btn.click(respond, [msg, chatbot], [msg, chatbot])
    clear_btn.click(clear_conversation, outputs=chatbot)

# Launch the interface
print("Launching modern Gradio interface...")
demo.launch(share=True, show_error=True)

## 5. Exercise: Advanced Geoscience Conversations

Now let's test our modern chat agent with complex geoscience scenarios:

In [ ]:
# Exercise 5: Advanced Geoscience Conversations with Galactica Model
model_name = "GeorgiaTechResearchInstitute/galactica-6.7b-evol-instruct-70k"

print(f"Loading Galactica model: {model_name}")

# Create tokenizer for Galactica

""" YOUR CODE HERE """

# Updated quantization config for the larger model

""" YOUR CODE HERE """

# Load the Galactica model

""" YOUR CODE HERE """

# Set pad token for Galactica
""" YOUR CODE HERE """

# Create pipeline for Galactica
""" YOUR CODE HERE """

# Create LangChain components for Galactica
galactica_llm = """ YOUR CODE HERE """
galactica_chat_model = ChatHuggingFace(llm=galactica_llm)

print(f"Galactica model loaded successfully!")
print(f"Model parameters: {galactica_model.num_parameters():,}")

In [ ]:
# Enhanced Geoscience Chat Agent for Advanced Scientific Queries
class AdvancedGeoscienceChatAgent(ModernGeoscienceChatAgent):
    def __init__(self, chat_model):
        super().__init__(chat_model)
        
        # Enhanced system prompt for scientific rigor with Galactica
        self.system_prompt = """
        YOUR PROMPT HERE
"""

# Create the advanced agent with Galactica
""" YOUR CODE HERE """
print("Advanced GeoscienceChatAgent with Galactica model created!")

In [ ]:
# Advanced Scientific Query Categories
advanced_scenarios = {
    "Quantitative Reservoir Analysis": [
        "Calculate the hydrocarbon pore volume for a reservoir with 25% porosity, 65% water saturation, and net pay of 150 ft over 1000 acres.",
        "What is the relationship between permeability and grain size in sandstone reservoirs? Provide the Kozeny-Carman equation.",
        "How do you estimate original oil in place using volumetric methods? Include uncertainty analysis."
    ],
    
    "Advanced Seismic Interpretation": [
        "Explain the physics behind AVO (Amplitude Versus Offset) analysis and the Zoeppritz equations.",
        "How do you distinguish between structural and stratigraphic hydrocarbon traps using seismic attributes?",
        "What are the key parameters in seismic resolution analysis? Provide the mathematical relationships."
    ],
    
    "Geomechanics & Drilling": [
        "Calculate the minimum horizontal stress using the poroelastic theory. Include Biot's coefficient.",
        "How do you predict wellbore stability using the Mohr-Coulomb failure criterion?",
        "What factors control hydraulic fracture propagation in unconventional reservoirs?"
    ],
    
    "Carbon Sequestration": [
        "What are the key thermodynamic properties of CO2 at typical reservoir conditions (3000 ft depth, 120°F)?",
        "How do you assess caprock integrity for CO2 storage? Include geochemical considerations.",
        "Calculate the CO2 storage capacity using the methodology from the CO2 Storage Atlas."
    ],
    
    "Machine Learning in Geosciences": [
        "How can neural networks be applied to seismic facies classification? What are the key preprocessing steps?",
        "Explain the use of clustering algorithms for well log analysis and formation evaluation.",
        "What machine learning approaches are most effective for reservoir property prediction from seismic data?"
    ]
}

In [ ]:
def create_advanced_gradio_interface():
    """Create an advanced Gradio interface for scientific geoscience queries"""
    
    # Global session
    global advanced_session
    advanced_session = advanced_agent.create_new_session()
    
    def advanced_respond(message: str, history: List[Tuple[str, str]]) -> Tuple[str, List[Tuple[str, str]]]:
        global advanced_session
        
        if not message.strip():
            return "", history
        
        # Get response from advanced agent
        bot_response = advanced_agent.chat(message, advanced_session)
        history.append((message, bot_response))
        return "", history
    
    def clear_advanced_conversation() -> List[Tuple[str, str]]:
        global advanced_session
        advanced_agent.clear_memory(advanced_session)
        advanced_session = advanced_agent.create_new_session()
        return []
    
    def load_scenario_questions(scenario_name: str) -> str:
        """Load all questions from a scenario"""
        if scenario_name in advanced_scenarios:
            questions = advanced_scenarios[scenario_name]
            return "\n\n".join([f"Q{i+1}: {q}" for i, q in enumerate(questions)])
        return ""
    
    # Create the interface
    with gr.Blocks(
        title="Dr. GeoBot-Advanced - Scientific Geoscience AI",
        theme=gr.themes.Base(),
        css="""
        .gradio-container { background: linear-gradient(45deg, #1e3a8a, #1e40af) }
        .chat-message { background: rgba(255,255,255,0.95); border-radius: 10px; }
        """
    ) as advanced_demo:
        gr.Markdown("""
        # 🧬 Dr. GeoBot-Advanced - Scientific Geoscience AI
        ### *Powered by Galactica-6.7B for Advanced Scientific Reasoning*
        
        | **🔬 Advanced Capabilities** | **📊 Quantitative Analysis** | **🎯 Research Applications** |
        |---|---|---|
        | Complex equation derivation | Statistical reservoir analysis | ML/AI in geosciences |
        | Multi-physics modeling | Uncertainty quantification | Carbon sequestration |
        | Advanced interpretation | Geomechanical calculations | Unconventional resources |
        
        💡 *Ask detailed scientific questions with quantitative requirements*
        """)
        
        with gr.Row():
            with gr.Column(scale=2):
                chatbot = gr.Chatbot(
                    value=[],
                    height=600,
                    show_label=False,
                    bubble_full_width=False,
                    avatar_images=("🧑‍🔬", "🤖")
                )
                
                with gr.Row():
                    msg = gr.Textbox(
                        placeholder="Ask advanced geoscience questions (include specific parameters, equations, calculations)...",
                        show_label=False,
                        scale=5,
                        container=False
                    )
                    send_btn = gr.Button("🚀 Analyze", scale=1, variant="primary")
                
                with gr.Row():
                    clear_btn = gr.Button("🗑️ Clear Session", variant="secondary")
                    export_btn = gr.Button("📋 Export Chat", variant="secondary")
            
            with gr.Column(scale=1):
                gr.Markdown("### 🔬 Advanced Query Categories")
                
                scenario_dropdown = gr.Dropdown(
                    choices=list(advanced_scenarios.keys()),
                    label="Select Scientific Domain",
                    value=None
                )
                
                scenario_text = gr.Textbox(
                    label="Scenario Questions",
                    lines=8,
                    placeholder="Select a domain to see advanced questions..."
                )
                
                load_scenario_btn = gr.Button("📖 Load Questions", variant="secondary")
                
                gr.Markdown("""
                ### 💡 Quick Examples
                - *"Calculate fracture pressure using Eaton's method"*
                - *"Derive the Gardner equation for density prediction"*
                - *"Explain Gassmann fluid substitution theory"*
                - *"Model CO2 solubility in brine at reservoir conditions"*
                """)
        
        # Event handlers
        msg.submit(advanced_respond, [msg, chatbot], [msg, chatbot])
        send_btn.click(advanced_respond, [msg, chatbot], [msg, chatbot])
        clear_btn.click(clear_advanced_conversation, outputs=chatbot)
        
        scenario_dropdown.change(
            load_scenario_questions,
            inputs=scenario_dropdown,
            outputs=scenario_text
        )
        
        load_scenario_btn.click(
            lambda text: text,
            inputs=scenario_text,
            outputs=msg
        )
    
    return advanced_demo

# Create and launch the advanced interface
print("Creating Gradio Interface with Galactica...")
advanced_demo = create_advanced_gradio_interface()
advanced_demo.launch(share=True, show_error=True)

## Summary

In this module, we built a modern conversational AI agent using current LangChain best practices:

### What We Learned:

1. **Modern LangChain Architecture**:
   - ✅ LCEL (LangChain Expression Language) for chain composition
   - ✅ ChatModels and structured message handling
   - ✅ RunnableWithMessageHistory for conversation management
   - ✅ Proper session management and memory handling

2. **Advanced Features**:
   - ✅ Multi-session conversation support
   - ✅ Structured prompt templates with MessagesPlaceholder
   - ✅ Modern error handling and response parsing
   - ✅ Session-based memory management

3. **Geoscience Applications**:
   - ✅ Domain-specific expert persona (Dr. GeoBot)
   - ✅ Technical geoscience conversation scenarios
   - ✅ Context-aware follow-up questions
   - ✅ Multi-topic expertise coverage

4. **Modern UI/UX**:
   - ✅ Enhanced Gradio interface with themes
   - ✅ Example question buttons for easy interaction
   - ✅ Improved conversation display and management
   - ✅ Session management and conversation clearing

### Key Improvements Over Legacy LangChain:

| **Legacy** | **Modern** |
|---|---|
| `LLMChain` | LCEL (`|` operator) |
| `ConversationBufferMemory` | `RunnableWithMessageHistory` |
| Manual prompt formatting | `ChatPromptTemplate` |
| Basic error handling | Structured exception management |
| Single conversation | Multi-session support |

### Next Steps:

- **Module 1.3**: Add RAG (Retrieval Augmented Generation) for factual accuracy
- **Module 1.4**: Integrate external tools and function calling
- **Session 2**: Fine-tune models on geoscience datasets
- **Session 3**: Build specialized applications (seismic analysis, log interpretation)

### Exercise Extensions:

1. **Customize the Expert**: Modify the system prompt to create specialists (seismic interpreter, reservoir engineer, etc.)
2. **Add Validation**: Implement response quality checking and topic relevance
3. **Export Conversations**: Add functionality to save/load conversation sessions
4. **Multi-Agent Setup**: Create multiple specialized agents for different domains
5. **Integration**: Connect with geoscience APIs or databases for real-time data

This modern implementation provides a solid foundation for building production-ready geoscience chat applications! 🌍